In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
skip_header=True
class RNN(nn.Module):
    def __init__(self, emb_dim_L, hid_dim_L, emb_dim_M, hid_dim_M):
        super(RNN, self).__init__()
        self.rnn_L = nn.LSTM(input_size=emb_dim_L,
                           hidden_size=hid_dim_L,
                           num_layers=2,
                           bidirectional=False)
        
        self.rnn_M = nn.LSTM(input_size=emb_dim_M,
                           hidden_size=hid_dim_M,
                           num_layers=2,
                           bidirectional=False)
        
        self.fc_L = nn.Linear(hid_dim_L, 128)
        self.fc_M = nn.Linear(hid_dim_M, 128)


    def forward(self, lyrics, melody):
        output_L, (hidden_L, cell) = self.rnn_L(lyrics)
        output_M, (hidden_M, cell) = self.rnn_M(melody)
        final_output_L = self.fc_L(hidden_L[-1])
        final_output_M = self.fc_M(hidden_M[-1])
    
        return final_output_L, final_output_M



model_LM = RNN(50, 256, 1, 256)


def binary_accuracy(preds, y):
    preds = torch.round(torch.sigmoid(preds))
    correct = (preds==y).float()
    acc = correct.sum() / len(correct) 
    return acc
print (model_LM)
# tensor1.reshape((457,50,1)).shape
LR = 1e-3

#create random dataset
train_data_combined = [[torch.randn(300, 1, 50),torch.randn(300, 1, 1),1],[torch.randn(300, 1, 50),torch.randn(300, 1, 1),0],[torch.randn(300, 1, 50),torch.randn(300, 1, 1),1]]
dev_data_combined = [[torch.randn(300, 1, 50),torch.randn(300, 1, 1),1],[torch.randn(300, 1, 50),torch.randn(300, 1, 1),0],[torch.randn(300, 1, 50),torch.randn(300, 1, 1),1]]

def loss_fn(feature1, feature2, label):
    similarity = cos(feature1, feature2)
    #print (similarity.shape, label, feature1.shape, feature2.shape)
    loss = nn.MSELoss() #loss 可改
    return loss(similarity, label)

def train(model_LM, train_data_combined, epoch):

    optimizer = optim.Adam(model_LM.parameters(), lr=LR) #list object has no attribute parameters后面的train也是一样
    
#     loss_fn = nn.BCEWithLogitsLoss(loss, label)
    
    epoch_loss = 0
    
    model_LM.train()
    for batch_idx, batch in enumerate(train_data_combined):
        lyrics = batch[0]#.lyrics
        note = batch[1]#.note
        label = batch[2]#.label
        label = torch.autograd.variable(label).float()
        feature_L, feature_M = model_LM(lyrics, note)
        optimizer.zero_grad()
        
        loss = loss_fn(feature_L, feature_M, label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

        
        if batch_idx % 200 == 0:
            print ("Epoch: {}, Batch Idx: {}, Loss: {}".format(epoch, batch_idx, loss.item()))
        return epoch_loss/len(train_data_combined)
    
    

def evaluate(model_LM, dev_data_combined):

    epoch_acc = 0
    
    #model_LM.test()
    for batch_idx, batch in enumerate(dev_data_combined):
        lyrics = batch[0]#.lyrics
        note = batch[1]#.note
        label = batch[2]#.label
        
        feature_L, feature_M = model_LM(lyrics, note) #text_lengths).squeeze(1)
        
        pred = (cos(feature_L, feature_M) > 0.6)
        
        acc = sum(pred == label)*1.0/len(dev_data_combined)
        
        epoch_acc += acc.item()
        return epoch_acc/len(dev_data_combined)


def epoch_time(start_time, end_time):
        elapsed_time = end_time - start_time
        mins = int(elapsed_time / 60)
        secs = int(elapsed_time - (mins)*60)
        print ("Time: {} mins {} secs".format(mins, secs))

EPOCH = 50

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
for epoch in range(1, EPOCH+1):
        start_time = time.time() 
    
        train_loss = train(model_LM, train_data_combined, epoch)
        
        dev_acc = evaluate(model_LM, dev_data_combined)
        end_time = time.time()
        epoch_time(start_time, end_time)
        print ("Epoch: {}, Acc: {}, Train loss: {}".format(epoch, dev_acc, train_loss))

# save model
SAVE_PATH = "matcher_rnn.ckpt"
torch.save(model_LM.state_dict(), SAVE_PATH)

RNN(
  (rnn_L): LSTM(50, 256, num_layers=2)
  (rnn_M): LSTM(1, 256, num_layers=2)
  (fc_L): Linear(in_features=256, out_features=128, bias=True)
  (fc_M): Linear(in_features=256, out_features=128, bias=True)
)
Epoch: 1, Batch Idx: 0, Loss: 1.2042142152786255
Time: 0 mins 0 secs
Epoch: 1, Acc: 0.0, Train loss: 0.4014047384262085
Epoch: 2, Batch Idx: 0, Loss: 0.1802227646112442
Time: 0 mins 0 secs
Epoch: 2, Acc: 0.11111111442248027, Train loss: 0.060074254870414734
Epoch: 3, Batch Idx: 0, Loss: 0.004099488724023104
Time: 0 mins 1 secs
Epoch: 3, Acc: 0.11111111442248027, Train loss: 0.0013664962413410346
Epoch: 4, Batch Idx: 0, Loss: 0.0142610352486372
Time: 0 mins 0 secs
Epoch: 4, Acc: 0.11111111442248027, Train loss: 0.0047536784162123995
Epoch: 5, Batch Idx: 0, Loss: 0.004343835171312094
Time: 0 mins 0 secs
Epoch: 5, Acc: 0.11111111442248027, Train loss: 0.0014479450571040313
Epoch: 6, Batch Idx: 0, Loss: 0.010761500336229801
Time: 0 mins 1 secs
Epoch: 6, Acc: 0.11111111442248027, Trai

In [ ]:
# load the model 
# model = RNN(emb_dim, hid_dim)
# model.load_state_dict(torch.load(SAVE_PATH))